In [3]:
import os
import sys
from argparse import ArgumentParser
from tqdm import tqdm, trange
import h5py as h5

import numpy as np
import torch
import utils

In [4]:
k_nn = 1000

In [26]:
train_dataset = utils.get_dataset_hdf5(
        **{
            "resolution": 256,
            "data_path": "/Work2/Watch_This/ICGAN/ic_gan/data",
            "load_in_mem_feats": True,
            "compute_nns": True,
            "longtail": False,
            "split": 'train',
            "instance_cond": True,
            "feature_extractor": "selfsupervised",
            "backbone_feature_extractor": "resnet50",
            "k_nn": k_nn,
            "ddp": False,
            "which_dataset": "imagenet",
            "test_part": False,
        }
    )

Load in mem? Images: False, Labels: True, Features: True.
Loading labels from /Work2/Watch_This/ICGAN/ic_gan/data/ILSVRC256_xy.hdf5 into memory...
Loading features from /Work2/Watch_This/ICGAN/ic_gan/data/ILSVRC256_feats_selfsupervised_resnet50.hdf5 into memory...
Normalizing features by their norm
HERE
HERE
HERE
self.num_imgs 1024
HERE
HERE
HERE


RuntimeError: Error in virtual void* faiss::gpu::StandardGpuResourcesImpl::allocMemory(const faiss::gpu::AllocRequest&) at /project/faiss/faiss/gpu/StandardGpuResources.cpp:452: Error: 'err == cudaSuccess' failed: StandardGpuResources: alloc fail type TemporaryMemoryBuffer dev 0 space Device stream 0x63c494e0 size 1610612736 bytes (cudaMalloc error out of memory [2])


In [ ]:
len(train_dataset)

In [13]:
all_nns = np.array(train_dataset.sample_nns)[:, : k_nn]
all_nns_radii = train_dataset.kth_values[:, k_nn]
print("NNs shape ", all_nns.shape, all_nns_radii.shape)
labels_ = torch.Tensor(train_dataset.labels)

NNs shape  (1024, 1000) (1281167,)


In [19]:
train_dataset.labels.shape

(1024,)

In [18]:
acc = np.array(
    [(labels_[all_nns[:, i_nn]] == labels_).sum() for i_nn in range(k_nn)]
).sum() / (len(labels_) * k_nn)
print("For k ", config["k_nn"], " accuracy:", acc)

IndexError: index 1192 is out of bounds for dimension 0 with size 1024